In [4]:
import subprocess
import os
import numpy as np
import re
from Bio import SeqIO

### Gathering the genomes with their names associated

Dict : GCF -> (Specie name,fnaPath,faaPath)

In [33]:
def DownloadingSequences(bacteriaPath,archeaPath,outputPath):
    
    print("Downloading Bacteria Files")
    subprocess.run("ncbi-genome-download --genera \'"+bacteriaPath+"\' bacteria -F protein-fasta,fasta -o "+outputPath,shell=True)
    
    print("Downloading Archaea Files")
    subprocess.run("ncbi-genome-download --genera \'"+archeaPath+"\' archaea -F protein-fasta,fasta -o "+outputPath,shell=True)
        
    CleaningFolder(outputPath)	

def CleaningFolder(path):
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith('.gz'):
                subprocess.run("gunzip "+root+"/"+file,shell=True)
            if file=="MD5SUMS":
                subprocess.run("rm "+root+"/"+file,shell=True)
            
            
DownloadingSequences('../../data/Bacteria.list','../../data/Archea.list','../../data/')

In [32]:
def ParsingSequences(path):
    
    #Getting the species names via Regular expression
    regex="[0-9]+ ([a-zA-Z 1-9]+),"
    
    listPhylums=os.listdir(path)
    listFiles=[[] for _ in range(len(listPhylums))]
    for i in range(len(listPhylums)):
        listfaa=[]
        listfna=[]
        for root,dirs,files in os.walk(path+'/'+listPhylum[i]):
            for file in files:
                if file.endswith('.faa'):
                    listfaa.append(path+'/'+file)
                elif file.endswith('.fna'):
                    listfna.append(path+'/'+file)
                    lineSpecie=open(root+'/'+file).readlines()[0]
                    print(lineSpecie)
                    m=re.search(regex,lineSpecie)
                    print(m)
                    
        listFiles[i]=list(zip(listfaa,listfna))
    return listFiles
                    
        

In [33]:
pathGenomes='../../data/refseq/'

listPhylum= os.listdir(pathGenomes)
listArchaea=[i[0:-1] for i in open('../../data/Archea.list','r').readlines()[1:]]
listbacteria=[i[0:-1] for i in open('../../data/Bacteria.list','r').readlines()[1:-2]]
listAdresses=ParsingSequences(pathGenomes)


>NC_003901.1 Methanosarcina mazei Go1, complete sequence

<re.Match object; span=(11, 38), match='1 Methanosarcina mazei Go1,'>
>NC_003413.1 Pyrococcus furiosus DSM 3638, complete sequence

<re.Match object; span=(11, 42), match='1 Pyrococcus furiosus DSM 3638,'>
>NC_008212.1 Haloquadratum walsbyi DSM 16790, complete genome

None
>NC_003551.1 Methanopyrus kandleri AV19, complete sequence

<re.Match object; span=(11, 40), match='1 Methanopyrus kandleri AV19,'>
>NZ_CP039138.1 Haloarcula marismortui ATCC 43049 chromosome, complete genome

<re.Match object; span=(43, 60), match='43049 chromosome,'>
>NC_007796.1 Methanospirillum hungatei JF-1, complete sequence

None
>NC_013790.1 Methanobrevibacter ruminantium M1, complete sequence

<re.Match object; span=(11, 47), match='1 Methanobrevibacter ruminantium M1,'>
>NC_000854.2 Aeropyrum pernix K1, complete genome

<re.Match object; span=(11, 33), match='2 Aeropyrum pernix K1,'>
>NZ_CP023154.1 Pyrococcus furiosus DSM 3638 chromosome, complete ge

>NZ_LT976985.1 Cupriavidus taiwanensis isolate Cupriavidus taiwanensis STM 8560 chromosome CBM2617_a, whole genome shotgun sequence

None
>NC_011297.1 Dictyoglomus thermophilum H-6-12, complete sequence

None
>NC_007204.1 Psychrobacter arcticus 273-4, complete sequence

None
>NC_007643.1 Rhodospirillum rubrum ATCC 11170, complete sequence

None
>NC_021214.1 Thermotoga maritima MSB8, complete sequence

<re.Match object; span=(11, 38), match='1 Thermotoga maritima MSB8,'>
>NZ_AZNS01000034.1 Campylobacter jejuni subsp. jejuni 81-176-UMCW7 plasmid pVir, whole genome shotgun sequence

<re.Match object; span=(65, 80), match='7 plasmid pVir,'>
>NZ_CP012716.1 Fusobacterium nucleatum subsp. nucleatum ChDC F316 strain KCOM 1322 chromosome, complete genome

<re.Match object; span=(62, 94), match='316 strain KCOM 1322 chromosome,'>
>NZ_UYXF01000001.1 Bacteroides fragilis YCH46 isolate Bacteroides fragilis 82A12, whole genome shotgun sequence

<re.Match object; span=(17, 81), match='1 Bacteroides f

>NZ_LT976995.1 Cupriavidus taiwanensis isolate Cupriavidus taiwanensis STM 8564 chromosome CBM2621_a, whole genome shotgun sequence

None
>NZ_ABGO01000922.1 Gemmata obscuriglobus UQM 2246 gcontig_1106221710807, whole genome shotgun sequence

None
>NC_005125.1 Gloeobacter violaceus PCC 7421, complete genome

<re.Match object; span=(11, 44), match='1 Gloeobacter violaceus PCC 7421,'>
>NC_017448.1 Fibrobacter succinogenes subsp. succinogenes S85, complete sequence

None
>NZ_KI912098.1 Rhizobium leguminosarum bv. trifolii CC278f Rleg6DRAFT_RLE.1, whole genome shotgun sequence

None
>NZ_LT992572.1 Cupriavidus taiwanensis isolate Cupriavidus taiwanensis mpp 1.1 chromosome CBM2626_a, whole genome shotgun sequence

None
>NC_010644.1 Elusimicrobium minutum Pei191, complete sequence

<re.Match object; span=(11, 43), match='1 Elusimicrobium minutum Pei191,'>
>NZ_CP053205.2 Rhizobium leguminosarum bv. trifolii TA1 chromosome, complete genome

<re.Match object; span=(54, 67), match='1 chromosome,'>